In [2]:
from ollama import chat
import time
import pandas as pd

def generate_sentences(label, output_file, number, display=False): 
    response = chat(
        model='llama3.2',
        messages=[
            {'role': 'user', 'content': f'You are generating {number} sentences about booking a {label}.'}],
        stream=True,
    )
    with open(output_file, 'w') as f:
        for chunk in response:
            if display:
                print(chunk['message']['content'], end='', flush=True)
            else:
                f.write(chunk['message']['content'])


In [21]:
generate_sentences('car rental', 'car.txt', 1000)

In [22]:
generate_sentences('flight', 'flight.txt', 1000)

In [23]:
generate_sentences('hotel', 'hotel.txt', 1000)

In [23]:
df = pd.DataFrame(columns=['sentence', 'label'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  0 non-null      object
 1   label     0 non-null      object
dtypes: object(2)
memory usage: 132.0+ bytes


In [30]:
with open('car.txt', 'r') as f:
    for line in f:
        sentence = line.split('. ')[1]
        df = pd.concat([df, pd.DataFrame(data={'sentence': [sentence], 'label': ['CarRental']})])

In [31]:
with open('flight.txt', 'r') as f:
    for line in f:
        sentence = line.split('. ')[1]
        df = pd.concat([df, pd.DataFrame(data={'sentence': [sentence], 'label': ['Flight']})])

In [32]:
with open('hotel.txt', 'r') as f:
    for line in f:
        sentence = line.split('. ')[1]
        df = pd.concat([df, pd.DataFrame(data={'sentence': [sentence], 'label': ['Hotel']})])

In [38]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
sentences = list(df['sentence'])
embeddings = model.encode(sentences)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\Stefan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Stefan\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [41]:
df_embeddings = pd.DataFrame(data=embeddings)

In [64]:
combined_df = pd.concat([df, df_embeddings], axis=1)

In [68]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(combined_df, test_size=0.2, random_state=42, shuffle=True)

val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [70]:
train_df.to_csv('train.csv')
test_df.to_csv('test.csv')
val_df.to_csv('val.csv')